In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [4]:
#FOURSQUARE IDs 
CLIENT_ID = '3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML' # your Foursquare ID
CLIENT_SECRET = 'V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

## Capstone Final Project 

### Subject: Social places vs Covid19 
Question I want to address is : Can we see relationship between large cities' social places (such as  bar, restaurant, cinema, theatre, ... ) and postive Covid19 cases in France ?
How to: I'm going to get  the list of biggest cities on France, get the numbers about the Covid19 cases in France, get the top most visited social places information from FourSquare for these cities. Aggregating these different data sets, I will leverage the clustering techniques to highlight similarities and hopefully get some insight of my original question. 
My assumption is that, the more we would have Social places the highest would be the number of Covid positive cases.. 


### Business problem
It is obvious I believe the economical impact of closing everything down. There are many different shopts being closed eventhough the medical protective measure were in place. Economical impact oculd be even bigger than the medical impact. Looking at the problem by mixing the most visited places and types of places vs the number of cases can shade some lights on what are the places we could open again.

### Data: 
I will gather  Covid related data: numbers of positive cases by region/department, number of Covid test  per region/departement to reduce the bias of number of positive vs number of tested.
<br>I will list of cities from which I will take the largest ( most representative ) for each department
<br>I will use the Foursquare API to get the "Social" places around the biggest cities 


#### Details on each data set to be used for this study:

#### Covid stats : pulling data from the governement portal

From May to December 2020
https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/
file: https://static.data.gouv.fr/resources/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/20201211-192020/sp-pos-quot-dep-2020-12-11-19h20.csv
url pointing to daily updates ( permanent url for latest info ): https://www.data.gouv.fr/fr/datasets/r/406c6a23-e283-4300-9484-54e78c8ae675
where:
- dep	is the departement ,
- jour is the date of data collection 	
- P	is positive tested results of the given date
- T	is tested in total of the given date
- cl_age90	: this attributes is not clearly explain I will then ignore it.
- pop is the total  population in that departement 


#### French Cities  list per departments: : pulling data from the governement portal

https://www.data.gouv.fr/fr/datasets/regions-departements-villes-et-villages-de-france-et-doutre-mer/
==> direct link to the data csv data set:https://www.data.gouv.fr/fr/datasets/r/a8adb872-dfac-46b1-9885-da8ca7d2d931, from where I will use the cities.csv 
where the below attributes are:
- id : unique ID
- department_cod : numeric code of the department where the city is
- insee_code : code being used for statistic city identifier
- zip_code : Postal code of the city
- name : Name of the department 
- slug : Name of the department without special character inside
- gps_lat	gps_lng  : coordinates of the City

largest cities (  based on population ):
https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peupl%C3%A9es
I'll take only the Year 2017 as it is the most recent 
From that url, I will use the table  named 'Communes de plus de 30 000 habitants' , and I will used the INSEE code to merge de data set with the previous data set so I can restrict my study to largest cities only.


#### Crisis Contextual information , to bring more insights 
Differents action in lockdown situation in France. I may input these information to refine observation and see if there are drops in the number of positive Covid tests
https://www.lemonde.fr/les-decodeurs/article/2020/05/12/coronavirus-de-la-chauve-souris-au-deconfinement-la-chronologie-de-la-pandemie_6039448_4355770.html
https://fr.wikipedia.org/wiki/Chronologie_de_la_pand%C3%A9mie_de_Covid-19_en_France
I will use these information on a visual representation to see the different influence ( or not ) of the lockdown situation on the positive tests, assuming I have enough data to cross at the same period.



## Read the Covid data

In [ ]:
import io
import requests

covid_url = "https://www.data.gouv.fr/fr/datasets/r/406c6a23-e283-4300-9484-54e78c8ae675"

src = requests.get(covid_url).content  # get raw formatted content data from that url
covid_data = pd.read_csv(io.StringIO(src.decode('utf-8')),sep=';',low_memory=False) # pass to pandas read_csv a 'file-like' object the raw we got from prev step

## Convert the jour to a known date formate for later use
covid_data.jour = pd.to_datetime(covid_data.jour)

##Calculate here the ratio Postive vs Tested to have a relative value to compare afterwards
covid_data['PtoT_ratio'] = covid_data['P']/covid_data['T']
covid_data['PtoT_ratio'].fillna(0,inplace=True) # put 0 whenever there is no data available in P or T
## Calculate positive vs the population of a given Department 
covid_data['PtoPop_ratio'] = covid_data['P']/covid_data['pop']

# This data set contains all the data related to Covid tests since May 2020
display (covid_data.tail())

# How many departement do we have:
print (" We have ",len(covid_data.dep.unique())," département in Covid dataset")
departement_covid = covid_data.dep.unique()


### Refine this dataset to keep only the average of PtoPop_ratio, as this is the data we want to  include in our study . Meaning what is the average Positive cases detection in regards to the Social places
covid_data_refine = covid_data.groupby(['dep']).agg({'PtoT_ratio':'mean'}).reset_index()

display (covid_data_refine.tail())

In [ ]:
## Last date entry from my Covid data source 
display ('Latest data entry date we have in the covid data ? ')
display (covid_data['jour'].unique()[-1])


## Get the coordinate of cities in France
### this dataset have been taken from the zip availabile in https://www.data.gouv.fr/fr/datasets/r/a8adb872-dfac-46b1-9885-da8ca7d2d931 , zip file which contains other file so I extracted that specific one

In [ ]:
import io
import requests

localfile = "C:/Users/JULIENSauvanet/Documents/JSAUVANET/Coursera/Projects/Coursera_Capstone/cities.csv"
french_cities_coordinates = pd.read_csv(localfile,sep=',',low_memory=False) # pass to pandas read_csv a 'file-like' object the raw we got from prev step . 

## Format the zip code to int
## Before I need to take care of those without Zip code from the TOM/DOM
french_cities_coordinates.zip_code.fillna('00000',inplace=True)
french_cities_coordinates.zip_code.astype(np.int64)

french_cities_coordinates.dtypes
display (french_cities_coordinates.head())
# How many departement do we have:
print (" We have ",len(french_cities_coordinates.department_code.unique())," département in City Coordinates dataset")

## Get the dataset to providing the largest cities in France

In [ ]:
url = "https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peupl%C3%A9es"
tables = pd.read_html(url, encoding='utf-8') # Returns list of all tables on page
# First table of that page is what we need .. 
big_french_cities = tables[0] 

# Cleaning the old Population statscolumns as I need only the most recent year ( 7th columns)
nbofcols= big_french_cities.shape[1]
col = 0
while col < 7:
    nbofcols = nbofcols - 1
    big_french_cities.drop(big_french_cities.columns[nbofcols], axis='columns',inplace=True)
    big_french_cities.shape[1]
    col = col +1

## Cleaning the Multiu Columns level
big_french_cities.columns = big_french_cities.columns.droplevel(1)

## Cleaning the Population numbers format to get rid of the x08F ==> str.replace('[^\x00-\x8F]', '')
big_french_cities['Population légale'] = big_french_cities['Population légale'].str.replace('[^\x00-\x8F]', '')
## get rid of non standard format in Population légale , found sometime there are info into brakets.. need to clean this up
big_french_cities['Population légale'] = big_french_cities['Population légale'].str.replace(r"\(.*\)","")
big_french_cities['Population légale'] = big_french_cities['Population légale'].astype(np.int64)

#print (big_french_cities.dtypes)
display (big_french_cities.tail())

# Do we have one "big" city for each department ?
print (" We have ",len(big_french_cities['Département'].unique())," département in Big cities dataset")

departement_big_french_cities = big_french_cities['Département'].unique()

In [ ]:
print ("Merge two data sets: largest cities and list of cities with their coordinates ( based on Insee Code)" )
## big_french_cities => CodeInsee & french_cities_coordinates => insee_code
# rename header for the merge operation
big_french_cities.rename(index=str, columns={"CodeInsee": "insee_code"}, inplace = True)

# merge our datafile wth the coordinate dataframe
fr_cities = pd.merge(big_french_cities, french_cities_coordinates, on='insee_code', how='inner') # Join info on PostalCode which is our unique value, common to both dataframe

# remove duplicates if any - I've found that multiple lines appears for same Commune with with different gps coordinates ( having multiple zip code within same City). which is useless in our case
fr_cities.drop_duplicates(subset=['insee_code','Commune','Département','department_code'],inplace=True)

#########################################
#########################################
#########################################
#########################################
#########################################
#Restrict the list for testing purposes
#tmp = fr_cities.loc[(fr_cities.Commune.isin(['Nice','Paris','Marseille','Lille']))] ##Limiting to Nice for now to preserve FourSquare calls while testing
#fr_cities = tmp
#########################################
#########################################
#########################################
#########################################

# drop the unwanted column
fr_cities = fr_cities.drop(['Rang2020','Statut','id'],axis=1)

print (" I have a list of " + str(fr_cities.shape[0]) + " cities, I need only the biggest cities per departement.")
print (" This is concerning " + str(len(fr_cities['Département'].unique())) + " departement.")

# keep only the largest city for each department :
# by doing: sort by Population légale and drop duplicate keep only first occurence :
tmp = fr_cities.sort_values(by=['Population légale'],ascending=False)
fr_cities = tmp.drop_duplicates(subset=['Département'], keep='first')

print (" I have now a reduced list of " + str(fr_cities.shape[0]) + " cities, keeping largest per department.")
print (" This is concerning " + str(len(fr_cities['Département'].unique())) + " departement.")
display (fr_cities.tail())


fr_cities_departement = fr_cities['Département'].unique()


In [ ]:
## What departement are we missing : i.e where there is no "big cities". These departement cannot be included in this study as there is no official data source found for them
# Relation between Insee Code and Departement code is : Departement are the first digit of the INSEE code . e.g all Insee code starting with 06xxxx relates to 06 Departement code
# from this , search what INSEE code is missing in the big cities dataset
departement_big_french_cities = big_french_cities.insee_code.astype(str).str[0:2].unique()

display (HTML(" Departement which will are considered : <br> Others - as we don't have big city information on them - are not going to be included "))
print ([x for x in fr_cities_departement if x not in departement_big_french_cities])


## Gather the famous social places for each of our  largest cities in France

In [ ]:
## Initialize the Foursquare data:
CLIENT_ID = '3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML' # your Foursquare ID
CLIENT_SECRET = 'V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 80
# function to get the venues for each cities 
# cities are taken from fr_cities , based on gps_lat,gps_lng for each Commune
def getNearbyVenues(names, latitudes, longitudes, radius=20000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results ])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Commune', 
                  'Commune Latitude', 
                  'Commune Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    ## filter out the Venues category we don't want .. we want to keep only what we defined as social places in the below list
    
    ## JSA update 27 Dec : make the same study without filters on social places to see whether cluster observation are differents 
    ### JSA update 27 Dec social_places = ['Restaurant','Coffee','Café','Tea Room','Bar','Bistro','Cinema']
    ### JSA update 27 Dec nearby_venues = nearby_venues.loc[nearby_venues['Venue Category'].str.contains('|'.join(social_places))]
    
    return(nearby_venues)


# Call the function for all our Cities and get back a dataframe with all the Venues
fr_cities_venues = getNearbyVenues(names=fr_cities['Commune'],
                                   latitudes=fr_cities['gps_lat'],
                                   longitudes=fr_cities['gps_lng']
                                  )

display (fr_cities_venues.head())

###  I'm now having the count of social places within the given radius of my biggest cities. 

data set <b>fr_cities_venues_grouped</b> is containing the list of Commune, with their coordinates and the number of Social places found

In [ ]:
#print ("What are the venue category we have " )
#print (fr_cities_venues['Venue Category'].unique())

# Count Social Venues per City
print ("Number of venues per largest city in each departement  " )
fr_cities_venues_grouped = fr_cities_venues.groupby(['Commune','Commune Latitude','Commune Longitude'])['Venue'].count().reset_index()
display (fr_cities_venues_grouped)
# Count per Venue category
print ("Count of venues per largest city in each departement - group by category " )
display (fr_cities_venues.groupby(['Commune','Commune Latitude','Commune Longitude','Venue Category']).count())


## Include  Covid data and merge them with the Cities data
#### To be able to have a visual representation of the Covid data, I'm going to use the largest City of each department , as the Covid information are based on department. 
#### This is the full data set I'm going to use for clustering analysis

In [ ]:
#display (covid_data.head())
#display (fr_cities.head())

print ("Merge two data sets: Cities data set and Covid data( based on Department 2 digits number )" )
## covid_data => dep & fr_cities => Département
# merge our datafiles with the coordinate dataframe
covid_city_data = pd.merge(covid_data_refine, fr_cities, left_on='dep', right_on='department_code', how='inner') # Join info Departement info

#display (covid_city_data)

## I then keep only the largest City of each department as representative data
myminpop = 10000
print ("covid_city_data before drop ",covid_city_data.shape)
covid_city_data.drop(covid_city_data[covid_city_data['Population légale'] < myminpop].index, inplace=True)
print ("covid_city_data AFTER drop ",covid_city_data.shape)
display (covid_city_data.head())

## Final data set build : to have all the data on a single data set:  Covid data , Cities and Venues informations
merge the fr_cities_venues_grouped with covid_city_data on the Commune name

In [ ]:
#print (fr_cities_venues_grouped.dtypes)
#print (covid_city_data.dtypes)

final_dataset = pd.merge(fr_cities_venues_grouped, covid_city_data, on='Commune', how='inner') # Join info Departement info
# cast the output format the covid data to keep less float digits 
pd.options.display.float_format = '{:,.4f}'.format

print (final_dataset.shape)
display (final_dataset)

#print (final_dataset.dtypes)

## display Covid data on a map to get a first sight

### The larger is the point on the map  the higher is the Postive/Test ratio ( Ratio built on mean value of all the Covid data available)


In [ ]:
# create map of New York using latitude and longitude values
latitude = 46.156782
longitude = 4.924699
map_france = folium.Map(location=[latitude, longitude], zoom_start=7) ## map centered on Paris coordinates

# add markers to map of France
for lat, lng, cityname,zip_code,covid in zip(final_dataset['gps_lat'], final_dataset['gps_lng'], final_dataset['Commune'], final_dataset['zip_code'],final_dataset['PtoT_ratio']):
    label = '{}, {}, {:,.3f}'.format(int(zip_code), cityname,covid)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=200*covid,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_france)
    
map_france


## Work with data set ( covid + cities pop + coordinates ) for clustering 
#### Clustering will be based on Cities having the most ptoPop_ratio and their social places 


In [ ]:
display (HTML('Normalize (OneHot encoding) the fr_cities_venues data to prepare for Kmean'))

# one hot encoding
fr_cities_venues_onehot = pd.get_dummies(fr_cities_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
fr_cities_venues_onehot['Commune'] = fr_cities_venues['Commune'] 

# move neighborhood column to the first column
fixed_columns = [fr_cities_venues_onehot.columns[-1]] + list(fr_cities_venues_onehot.columns[:-1])
fr_cities_venues_onehot = fr_cities_venues_onehot[fixed_columns]

#display (fr_cities_venues_onehot.head(25))
fr_cities_venues_onehot_grouped = fr_cities_venues_onehot.groupby('Commune').mean().reset_index()
fr_cities_venues_onehot_grouped.head(25)

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Commune']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
fr_cities_venues_sorted = pd.DataFrame(columns=columns)
fr_cities_venues_sorted['Commune'] = fr_cities_venues_onehot_grouped['Commune']

for ind in np.arange(fr_cities_venues_onehot_grouped.shape[0]):
    fr_cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(fr_cities_venues_onehot_grouped.iloc[ind, :], num_top_venues)

fr_cities_venues_sorted

In [ ]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

display (HTML('<h3>Run _k_-means to cluster the Commune into N clusters.</h3>'))

# set number of clusters
kclusters = 5

fr_cities_grouped_clustering = fr_cities_venues_onehot_grouped.drop('Commune', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(fr_cities_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

In [ ]:
# add clustering labels
try:
    fr_cities_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
except ValueError:
    print (" cluster labels already there you need to recreate the DF fr_cities_venues_sorted above !!  ")


#fr_cities_merged = final_dataset[['Commune','Commune Latitude','Commune Longitude','Venue','pop','PtoT_ratio','dep']]
fr_cities_merged = final_dataset[['Commune','Commune Latitude','Commune Longitude','Venue','PtoT_ratio','dep']]


# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
fr_cities_merged = fr_cities_merged.join(fr_cities_venues_sorted.set_index('Commune'), on='Commune')

fr_cities_merged # check the last columns!


In [ ]:
display (HTML('<h3>Visual the cluster in a map.</h3>'))

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, covid in zip(fr_cities_merged['Commune Latitude'], fr_cities_merged['Commune Longitude'], fr_cities_merged['Commune'], fr_cities_merged['Cluster Labels'], fr_cities_merged['PtoT_ratio']):
    label = '{}, Cluster {}, PtoT {:,.3f}'.format(str(poi), cluster,covid)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=200*covid,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters 

In [ ]:

cluster_list = fr_cities_merged['Cluster Labels'].unique()
for cluster in cluster_list:
    display (HTML('<h3>Cluster '+str(cluster)+'</h3>'))
    display (fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == cluster].agg({'PtoT_ratio':'mean'}).reset_index())
    display (fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == cluster])
